In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./kit_part.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_.drop_duplicates(['Kit_JOIN_TCPK'], keep='first').reset_index(drop=True)

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error, output_null
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                data = {'func': 'getbuyersguide',
                        'payload': '{"partData":{"listing_data_essential":{"parttype":"' + input_.loc[a, 'Kit_Type_Code'] + '","partkey":"' + input_.loc[a, 'Kit_Part_Key'] + '"}}}',
                        'api_json_request': '1'}
                
                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text

                # = = = = = = = = = = = = = = =

                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(json_data['buyersguidepieces']['body'], 'lxml')
                html = etree.HTML(str(soup))
                    
                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                list_tr = html.xpath('//div[contains(@class, "buyersguide-nested")]/div/table[contains(@class, "nobmp")]/tr')
                for tr in list_tr:
                    list_td = [text.strip() for text in tr.xpath('./td/text()')]
                    list_vehicle.append(' '.join(list_td).strip())

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'Kit_JOIN_TCPK': input_.loc[a, 'Kit_JOIN_TCPK'],
                                         'Kit_Vehcile': vehicle}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Kit_Type_Code': input_.loc[a, 'Kit_Type_Code'],
                                     'Kit_Part_Key': input_.loc[a, 'Kit_Part_Key'],
                                     'Kit_JOIN_TCPK': input_.loc[a, 'Kit_JOIN_TCPK']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Kit_Type_Code'] + ' - ' + input_.loc[a, 'Kit_Part_Key'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Kit_JOIN_TCPK'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./kit_vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Kit_Type_Code', 'Kit_Part_Key'], ascending=[True, True]).reset_index(drop=True)
    output_error.to_excel('./kit_vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：890

[ok] - 5736 - 10110032
[尝试次数：1] - [剩余数量：878] - [当前时间：13:46:11]

[ok] - 5724 - 10110012
[尝试次数：1] - [剩余数量：877] - [当前时间：13:46:11]

[ok] - 5723 - 8314716
[尝试次数：1] - [剩余数量：876] - [当前时间：13:46:11]

[ok] - 5724 - 10110008
[尝试次数：1] - [剩余数量：875] - [当前时间：13:46:11]

[ok] - 5736 - 10110036
[尝试次数：1] - [剩余数量：874] - [当前时间：13:46:11]

[ok] - 5736 - 10110024
[尝试次数：1] - [剩余数量：873] - [当前时间：13:46:12]

[ok] - 5728 - 10110020
[尝试次数：1] - [剩余数量：872] - [当前时间：13:46:12]

[ok] - 10465 - 10195124
[尝试次数：1] - [剩余数量：871] - [当前时间：13:46:12]

[ok] - 5728 - 10110016
[尝试次数：1] - [剩余数量：870] - [当前时间：13:46:12]

[ok] - 5736 - 8307912
[尝试次数：1] - [剩余数量：869] - [当前时间：13:46:12]

[ok] - 5724 - 7093400
[尝试次数：1] - [剩余数量：868] - [当前时间：13:46:12]

[ok] - 5728 - 8307288
[尝试次数：1] - [剩余数量：867] - [当前时间：13:46:12]

[ok] - 5736 - 8307904
[尝试次数：1] - [剩余数量：866] - [当前时间：13:46:13]

[ok] - 5736 - 8307908
[尝试次数：1] - [剩余数量：865] - [当前时间：13:46:13]

[ok] - 5723 - 8314756
[尝试次数：2] - [剩余数量：864] - [当前时间：13:46:13]

[ok] - 5736 - 10110028
[尝试次数：1] - [剩余

[ok] - 5736 - 10890344
[尝试次数：16] - [剩余数量：748] - [当前时间：13:46:57]

[ok] - 5736 - 8308072
[尝试次数：16] - [剩余数量：747] - [当前时间：13:46:57]

[ok] - 5736 - 8307932
[尝试次数：2] - [剩余数量：746] - [当前时间：13:46:57]

[ok] - 5724 - 8305964
[尝试次数：1] - [剩余数量：745] - [当前时间：13:46:57]

[ok] - 5728 - 10519396
[尝试次数：23] - [剩余数量：744] - [当前时间：13:46:58]

[ok] - 11860 - 8308296
[尝试次数：2] - [剩余数量：743] - [当前时间：13:46:59]

[ok] - 5736 - 8307876
[尝试次数：6] - [剩余数量：742] - [当前时间：13:47:00]

[ok] - 11859 - 8921276
[尝试次数：4] - [剩余数量：741] - [当前时间：13:47:00]

[ok] - 5728 - 8307892
[尝试次数：2] - [剩余数量：740] - [当前时间：13:47:00]

[ok] - 5728 - 8307276
[尝试次数：11] - [剩余数量：739] - [当前时间：13:47:00]

[ok] - 5723 - 8314776
[尝试次数：9] - [剩余数量：738] - [当前时间：13:47:01]

[ok] - 5724 - 11158037
[尝试次数：4] - [剩余数量：737] - [当前时间：13:47:01]

[ok] - 5728 - 11158197
[尝试次数：1] - [剩余数量：736] - [当前时间：13:47:01]

[ok] - 5736 - 8307936
[尝试次数：12] - [剩余数量：735] - [当前时间：13:47:02]

[ok] - 5728 - 8307896
[尝试次数：15] - [剩余数量：734] - [当前时间：13:47:02]

[ok] - 5728 - 11158177
[尝试次数：6] - [剩余数量：733

[ok] - 5736 - 14047841
[尝试次数：8] - [剩余数量：619] - [当前时间：13:47:50]

[ok] - 5568 - 13570021
[尝试次数：2] - [剩余数量：618] - [当前时间：13:47:50]

[ok] - 5728 - 14047801
[尝试次数：1] - [剩余数量：617] - [当前时间：13:47:51]

[ok] - 5724 - 14047797
[尝试次数：4] - [剩余数量：616] - [当前时间：13:47:51]

[ok] - 12910 - 14048021
[尝试次数：6] - [剩余数量：615] - [当前时间：13:47:52]

[ok] - 5723 - 14047933
[尝试次数：2] - [剩余数量：614] - [当前时间：13:47:52]

[ok] - 5564 - 8310168
[尝试次数：3] - [剩余数量：613] - [当前时间：13:47:53]

[ok] - 5736 - 14047813
[尝试次数：19] - [剩余数量：612] - [当前时间：13:47:53]

[ok] - 5728 - 14047825
[尝试次数：7] - [剩余数量：611] - [当前时间：13:47:53]

[ok] - 5589 - 12804309
[尝试次数：9] - [剩余数量：610] - [当前时间：13:47:54]

[ok] - 5564 - 8310164
[尝试次数：7] - [剩余数量：609] - [当前时间：13:47:54]

[ok] - 5723 - 14047949
[尝试次数：17] - [剩余数量：608] - [当前时间：13:47:55]

[ok] - 5564 - 13060809
[尝试次数：12] - [剩余数量：607] - [当前时间：13:47:55]

[ok] - 5722 - 14047957
[尝试次数：11] - [剩余数量：606] - [当前时间：13:47:55]

[ok] - 5564 - 8310284
[尝试次数：10] - [剩余数量：605] - [当前时间：13:47:56]

[ok] - 5722 - 8314832
[尝试次数：4] - [剩余数

[ok] - 18586 - 15583709
[尝试次数：6] - [剩余数量：490] - [当前时间：13:48:38]

[ok] - 18586 - 15583713
[尝试次数：8] - [剩余数量：489] - [当前时间：13:48:39]

[ok] - 5722 - 7308128
[尝试次数：4] - [剩余数量：488] - [当前时间：13:48:39]

[ok] - 5724 - 8306220
[尝试次数：4] - [剩余数量：487] - [当前时间：13:48:39]

[ok] - 5722 - 8313724
[尝试次数：2] - [剩余数量：486] - [当前时间：13:48:39]

[ok] - 5736 - 8307700
[尝试次数：3] - [剩余数量：485] - [当前时间：13:48:39]

[ok] - 5723 - 8314300
[尝试次数：1] - [剩余数量：484] - [当前时间：13:48:40]

[ok] - 5724 - 8305700
[尝试次数：3] - [剩余数量：483] - [当前时间：13:48:40]

[ok] - 5188 - 8304576
[尝试次数：16] - [剩余数量：482] - [当前时间：13:48:41]

[ok] - 5728 - 7084236
[尝试次数：8] - [剩余数量：481] - [当前时间：13:48:41]

[ok] - 5723 - 8314160
[尝试次数：5] - [剩余数量：480] - [当前时间：13:48:42]

[ok] - 5724 - 7315468
[尝试次数：2] - [剩余数量：479] - [当前时间：13:48:42]

[ok] - 5728 - 7093412
[尝试次数：7] - [剩余数量：478] - [当前时间：13:48:42]

[ok] - 5728 - 8307444
[尝试次数：22] - [剩余数量：477] - [当前时间：13:48:43]

[ok] - 5722 - 8313840
[尝试次数：4] - [剩余数量：476] - [当前时间：13:48:44]

[ok] - 5723 - 8313700
[尝试次数：4] - [剩余数量：475] - [当前

[ok] - 5722 - 8313860
[尝试次数：2] - [剩余数量：360] - [当前时间：13:49:19]

[ok] - 5723 - 8313836
[尝试次数：2] - [剩余数量：359] - [当前时间：13:49:20]

[ok] - 5728 - 9659100
[尝试次数：13] - [剩余数量：358] - [当前时间：13:49:20]

[ok] - 5736 - 8308288
[尝试次数：11] - [剩余数量：357] - [当前时间：13:49:20]

[ok] - 5723 - 8314648
[尝试次数：6] - [剩余数量：356] - [当前时间：13:49:20]

[ok] - 5722 - 8314068
[尝试次数：3] - [剩余数量：355] - [当前时间：13:49:21]

[ok] - 5723 - 8313764
[尝试次数：6] - [剩余数量：354] - [当前时间：13:49:21]

[ok] - 5724 - 8304644
[尝试次数：3] - [剩余数量：353] - [当前时间：13:49:21]

[ok] - 5724 - 8304680
[尝试次数：2] - [剩余数量：352] - [当前时间：13:49:21]

[ok] - 5723 - 8313736
[尝试次数：6] - [剩余数量：351] - [当前时间：13:49:21]

[ok] - 5723 - 8314608
[尝试次数：7] - [剩余数量：350] - [当前时间：13:49:22]

[ok] - 10465 - 8314480
[尝试次数：2] - [剩余数量：349] - [当前时间：13:49:22]

[ok] - 5728 - 8307172
[尝试次数：3] - [剩余数量：348] - [当前时间：13:49:23]

[ok] - 5723 - 8314464
[尝试次数：3] - [剩余数量：347] - [当前时间：13:49:23]

[ok] - 5724 - 8304580
[尝试次数：10] - [剩余数量：346] - [当前时间：13:49:24]

[ok] - 5724 - 8304696
[尝试次数：1] - [剩余数量：345] - [当前时间

[ok] - 5724 - 8305996
[尝试次数：3] - [剩余数量：230] - [当前时间：13:50:05]

[ok] - 5728 - 8307300
[尝试次数：2] - [剩余数量：229] - [当前时间：13:50:06]

[ok] - 5728 - 8307252
[尝试次数：4] - [剩余数量：228] - [当前时间：13:50:06]

[ok] - 5332 - 14047901
[尝试次数：4] - [剩余数量：227] - [当前时间：13:50:06]

[ok] - 5724 - 8305980
[尝试次数：15] - [剩余数量：226] - [当前时间：13:50:06]

[ok] - 5728 - 8307304
[尝试次数：4] - [剩余数量：225] - [当前时间：13:50:07]

[ok] - 5722 - 8314700
[尝试次数：2] - [剩余数量：224] - [当前时间：13:50:08]

[ok] - 5728 - 8307296
[尝试次数：7] - [剩余数量：223] - [当前时间：13:50:08]

[ok] - 5728 - 8307920
[尝试次数：1] - [剩余数量：222] - [当前时间：13:50:09]

[ok] - 5724 - 8304688
[尝试次数：11] - [剩余数量：221] - [当前时间：13:50:09]

[ok] - 5722 - 8314536
[尝试次数：14] - [剩余数量：220] - [当前时间：13:50:09]

[ok] - 5723 - 8314748
[尝试次数：11] - [剩余数量：219] - [当前时间：13:50:11]

[ok] - 5722 - 8314744
[尝试次数：15] - [剩余数量：218] - [当前时间：13:50:11]

[ok] - 5736 - 8307968
[尝试次数：8] - [剩余数量：217] - [当前时间：13:50:11]

[ok] - 5736 - 8307924
[尝试次数：12] - [剩余数量：216] - [当前时间：13:50:12]

[ok] - 5723 - 8314696
[尝试次数：14] - [剩余数量：215] - [

[ok] - 5728 - 9659132
[尝试次数：3] - [剩余数量：100] - [当前时间：13:50:55]

[ok] - 51028 - 9659032
[尝试次数：11] - [剩余数量：99] - [当前时间：13:50:55]

[ok] - 5728 - 9659068
[尝试次数：6] - [剩余数量：98] - [当前时间：13:50:55]

[ok] - 5724 - 8306588
[尝试次数：1] - [剩余数量：97] - [当前时间：13:50:56]

[ok] - 5728 - 8528008
[尝试次数：1] - [剩余数量：96] - [当前时间：13:50:57]

[ok] - 5736 - 9659136
[尝试次数：7] - [剩余数量：95] - [当前时间：13:50:57]

[ok] - 5589 - 9659236
[尝试次数：3] - [剩余数量：94] - [当前时间：13:50:57]

[ok] - 5728 - 8527980
[尝试次数：5] - [剩余数量：93] - [当前时间：13:50:57]

[ok] - 5724 - 7109056
[尝试次数：3] - [剩余数量：92] - [当前时间：13:50:58]

[ok] - 11859 - 9659072
[尝试次数：1] - [剩余数量：91] - [当前时间：13:50:58]

[ok] - 5723 - 9659244
[尝试次数：6] - [剩余数量：90] - [当前时间：13:50:59]

[ok] - 5736 - 8308300
[尝试次数：9] - [剩余数量：89] - [当前时间：13:50:59]

[ok] - 5723 - 7305180
[尝试次数：47] - [剩余数量：88] - [当前时间：13:50:59]

[ok] - 5723 - 9659240
[尝试次数：7] - [剩余数量：87] - [当前时间：13:50:59]

[ok] - 5728 - 8307208
[尝试次数：3] - [剩余数量：86] - [当前时间：13:51:00]

[ok] - 5728 - 8307024
[尝试次数：8] - [剩余数量：85] - [当前时间：13:51:01]

[ok